# DS-SF-34 | 03 | Databases, Scrapping, and APIs | Codealong | Starter Code

# Part A | RDBMS Databases and `SQLite`

In [ ]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [ ]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-03-zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [ ]:
pd.io.sql.read_sql(
'''
SELECT *
    FROM properties
    LIMIT 10
;
''', con = db)

### `SELECT`

In [ ]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

### `WHERE`

In [ ]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

> #### Here's let's convert the unit of `sale_price` from \$ to \$M

In [ ]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

### `UNION`

> #### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [ ]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

### `WITH`

In [ ]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

### `JOIN`

In [ ]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id
;
''', con = db)

### `WITH` (take 2)

In [ ]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM beds_normalized_sale_prices
;
''', con = db)

### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bedrooms

In [ ]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT AVG(sale_price) AS mean_sale_price
    FROM beds_normalized_sale_prices
    GROUP BY beds
;
''', con = db)

Note: `beds = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [ ]:
# TODO

> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [ ]:
# TODO